### Imports

In [1]:
import argparse
from inspect import currentframe, getframeinfo
from glob import glob
import numpy as np
import os
from os.path import join
import pandas as pd
from pathlib import Path
import pickle
import sys
import nibabel as nib
from nistats.first_level_model import FirstLevelModel
from nistats.reporting import plot_design_matrix, plot_contrast_matrix
from utils.firstlevel_utils import get_first_level_objs, make_first_level_obj, save_first_level_obj

/usr/local/miniconda/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/local/miniconda/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


### Parse Arguments
These are not needed for the jupyter notebook, but are used after conversion to a script for production

- conversion command:
  - jupyter nbconvert --to script --execute 1stlevel_analysis.ipynb

In [2]:
parser = argparse.ArgumentParser(description='First Level Entrypoint script')
parser.add_argument('-data_dir', default='/data')
parser.add_argument('-derivatives_dir', default=None)
parser.add_argument('-working_dir', default=None)
parser.add_argument('--subject_ids', nargs="+")
parser.add_argument('--tasks', nargs="+", help="Choose from ANT, CCTHot, discountFix, \
                                    DPX, motorSelectiveStop, stopSignal, \
                                    stroop, surveyMedley, twoByTwo, WATT3")
parser.add_argument('--n_procs', default=16, type=int)
parser.add_argument('--overwrite', action='store_true')
parser.add_argument('--quiet', '-q', action='store_true')

if '-derivatives_dir' in sys.argv or '-h' in sys.argv:
    args = parser.parse_args()
else:
    args = parser.parse_args([])
    args.data_dir = '/data'
    args.derivatives_dir = '/data/derivatives'
    args.tasks = ['stroop', 'stopSignal']
    args.subject_ids = ['s358']
    args.n_procs=1
    # get all subjects
    all_subjs = sorted([i[-4:] for i in glob(os.path.join(args.data_dir, '*')) if 'sub-' in i])
    args.subject_ids = all_subjs

In [3]:
if not args.quiet:
    def verboseprint(*args, **kwargs):
        print(*args, **kwargs)
else:
    verboseprint = lambda *a, **k: None # do-nothing function

### Initial Setup

In [4]:
# list of subject identifiers
subjects = args.subject_ids
# set task
if args.tasks is not None:
    tasks = args.tasks
else:
    tasks = ['ANT', 'CCTHot', 'discountFix',
            'DPX', 'motorSelectiveStop',
            'stopSignal', 'stroop',
            'twoByTwo', 'WATT3']

#### Experiment Variables
derivatives_dir = args.derivatives_dir
fmriprep_dir = join(derivatives_dir, 'fmriprep', 'fmriprep')
data_dir = args.data_dir
first_level_dir = join(derivatives_dir,'1stlevel')
if args.working_dir is None:
    working_dir = join(derivatives_dir, '1stlevel_workingdir')
else:
    working_dir = join(args.working_dir, '1stlevel_workingdir')
n_procs = args.n_procs
# TR of functional images
TR = .68

In [5]:
# print
verboseprint('*'*79)
verboseprint('Tasks: %s\n, Subjects: %s\n, derivatives_dir: %s\n, data_dir: %s' % 
     (tasks, subjects, derivatives_dir, data_dir))
verboseprint('*'*79)

*******************************************************************************
Tasks: ['stroop', 'stopSignal']
, Subjects: ['s061', 's130', 's144', 's172', 's192', 's234', 's251', 's358', 's373', 's445', 's465', 's471', 's483', 's491', 's495', 's497', 's499', 's512', 's513', 's518', 's519', 's524', 's525', 's526', 's533', 's541', 's546', 's548', 's549', 's553', 's554', 's555', 's556', 's557', 's558', 's561', 's567', 's568', 's570', 's572', 's573', 's574', 's577', 's579', 's581', 's582', 's583', 's584', 's585', 's586', 's587', 's588', 's589', 's590', 's591', 's592', 's593', 's594', 's595', 's596', 's597', 's598', 's599', 's600', 's601', 's602', 's603', 's604', 's605', 's606', 's607', 's608', 's609', 's610', 's611', 's612', 's613', 's614', 's615', 's616', 's617', 's618', 's619', 's621', 's622', 's623', 's624', 's626', 's627', 's628', 's629', 's631', 's633', 's634', 's635', 's636', 's637', 's638', 's639', 's640', 's641', 's642', 's643', 's644', 's645', 's646', 's647', 's648', 's649', 's6

# Set up Nodes

### Run analysis

In [6]:
to_run = []
for subject_id in subjects:
    for task in tasks:
        files = get_first_level_objs(subject_id, task, first_level_dir)
        if len(files) == 0 or args.overwrite:
            subjinfo = make_first_level_obj(subject_id, task, fmriprep_dir, data_dir, TR)
            if subjinfo is not None:
                to_run.append(subjinfo)

Missing MRI files for s144: stroop


/usr/local/miniconda/lib/python3.6/site-packages/nistats/experimental_paradigm.py:63: UserWarning: 'modulation' column found in the given events data.
  warnings.warn("'modulation' column found in the given events data.")
/usr/local/miniconda/lib/python3.6/site-packages/nistats/hemodynamic_models.py:268: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  frame_times.max() * (1 + 1. / (n - 1)), n_hr)
/usr/local/miniconda/lib/python3.6/site-packages/nistats/hemodynamic_models.py:55: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  time_stamps = np.linspace(0, time_length, float(time_length) / dt)


Missing MRI files for s518: stroop
Missing MRI files for s518: stopSignal
Missing MRI files for s533: stroop
Missing MRI files for s533: stopSignal
Missing event files for s592: stopSignal
Missing MRI files for s593: stroop
Missing MRI files for s593: stopSignal
Missing MRI files for s599: stroop
Missing MRI files for s599: stopSignal
Missing MRI files for s600: stroop
Missing MRI files for s600: stopSignal
Missing MRI files for s603: stroop
Missing MRI files for s604: stroop
Missing MRI files for s604: stopSignal
Missing MRI files for s609: stroop
Missing MRI files for s609: stopSignal
Missing MRI files for s613: stroop
Missing MRI files for s613: stopSignal
Missing MRI files for s614: stroop
Missing MRI files for s614: stopSignal
Missing MRI files for s615: stroop
Missing MRI files for s615: stopSignal
Missing MRI files for s616: stroop
Missing MRI files for s616: stopSignal
Missing MRI files for s617: stroop
Missing MRI files for s617: stopSignal
Missing MRI files for s618: stroop
M

### Run model fit

In [8]:
"""
compare to things I did before:
high pass filter (80hz)
model_serial_correlations
should I mask? Don't think it matters
"""

for subjinfo in to_run:
    print(subjinfo.ID)
    print('** fitting model')
    fmri_glm = FirstLevelModel(TR, 
                           subject_label = subjinfo.ID,
                           mask=subjinfo.mask,
                           noise_model='ar1',
                           standardize=False, 
                           hrf_model='spm',
                           drift_model='cosine',
                           period_cut=80,
                           n_jobs=1,
                           smoothing_fwhm=4.4, # remove when doing this for real?
                          )
    %time out = fmri_glm.fit(subjinfo.func, design_matrices=subjinfo.design)
    subjinfo.fit_model = out
    
    # run contrasts and save
    verboseprint('** computing contrasts')
    for name, contrast in subjinfo.contrasts:
        z_map = subjinfo.fit_model.compute_contrast(contrast, output_type='z_score')
        subjinfo.maps[name+'_zscore'] = z_map
    verboseprint('** saving')
    save_first_level_obj(subjinfo, first_level_dir)

s499_stroop
** fitting model
CPU times: user 16.2 s, sys: 3.37 s, total: 19.6 s
Wall time: 1min 35s
** computing contrasts
** saving
s499_stopSignal
** fitting model
CPU times: user 26.3 s, sys: 4.86 s, total: 31.1 s
Wall time: 2min 25s
** computing contrasts
** saving
s512_stroop
** fitting model
CPU times: user 17.1 s, sys: 3.07 s, total: 20.2 s
Wall time: 1min 44s
** computing contrasts
** saving
s512_stopSignal
** fitting model
CPU times: user 29.2 s, sys: 4.47 s, total: 33.6 s
Wall time: 2min 40s
** computing contrasts
** saving
s513_stroop
** fitting model
CPU times: user 16.4 s, sys: 3.03 s, total: 19.4 s
Wall time: 1min 32s
** computing contrasts
** saving
s513_stopSignal
** fitting model
CPU times: user 25.6 s, sys: 4.37 s, total: 30 s
Wall time: 2min 23s
** computing contrasts
** saving
s519_stroop
** fitting model
CPU times: user 16.1 s, sys: 2.99 s, total: 19.1 s
Wall time: 1min 30s
** computing contrasts
** saving
s519_stopSignal
** fitting model
CPU times: user 24.6 s, sy

OSError: [Errno 107] Transport endpoint is not connected

** computing contrasts


PatsyError: unrecognized token in constraint
    congruent
    ^